In [1]:
import os
import random
import pandas as pd
import numpy as np
import cv2
from keras.utils import np_utils

In [2]:
IMAGE_SIZE = 100
CLASS = ["angry","disgust","fear","happy","neutral","sad","surprise"]
train_dir = "images/train"
test_dir = "images/test"

In [3]:
training_data = []

In [4]:
def create_data(dir_name) :
    
    for cl in CLASS :
        cl_path = os.path.join(dir_name,cl)
        class_num = CLASS.index(cl)
        for img in os.listdir(cl_path) :
            try :
                img_array = cv2.imread(os.path.join(cl_path,img))
                img_array = cv2.resize(img_array,(IMAGE_SIZE,IMAGE_SIZE))
                gray_img  = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                training_data.append([gray_img, class_num])
            except :
                pass
            
        random.shuffle(training_data)
        
    #return data_set
            
        

In [5]:
create_data(train_dir)
create_data(test_dir)

In [6]:
#shuffle the data
random.shuffle(training_data)
print(len(training_data))

35887


In [7]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

In [8]:
X = np.array(X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
y = np.array(y)

X = X/255.0

In [9]:
num_classes = len(os.listdir(train_dir))
num_classes

7

In [10]:
X.shape[1:]

(100, 100, 1)

In [11]:
# convert class labels to one-hot encoding
y = np_utils.to_categorical(y, num_classes)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [13]:
def build_model(hp) :
    model = keras.Sequential([
                                Conv2D(
                                    filters = hp.Int('conv_1_filter', min_value=32,max_value=190, step=32),
                                    kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                                    activation='relu',input_shape=(100,100,1)
                                ),
                                MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
                                Conv2D(
                                    filters = hp.Int('conv_2_filter', min_value=64, max_value=256, step=32),
                                    kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                                    activation='relu'
                                ),
                                MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
                                Conv2D(
                                    filters = hp.Int('conv_3_filter', min_value=48, max_value=240, step=32),
                                    kernel_size=hp.Choice('conv_3_kernel',values=[3,5]),
                                    activation='relu'
                                ),
                                MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
                                keras.layers.Flatten(),
                                Dense(units=hp.Int('dense1_units', min_value=48, max_value=240, step=48), activation='relu'),
                                Dense(units=hp.Int('dense2_units', min_value=48, max_value=240, step=48), activation='relu'),
        
                                Dense(num_classes,activation='softmax')
                            ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate",values=[1e-2,1e-3])),
                    loss="categorical_crossentropy", metrics=["accuracy"])

    return model


In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_Search = RandomSearch(build_model, 
                            objective="val_accuracy", max_trials=4, 
                            directory="output", project_name="emotion_detection" )

INFO:tensorflow:Reloading Oracle from existing project output/emotion_detection/oracle.json


In [ ]:
tuner_Search.search(X,y, epochs=3, validation_split=0.2,verbose=1)

Trial 3 Complete [00h 32m 48s]
val_accuracy: 0.2581498920917511

Best val_accuracy So Far: 0.5100306272506714
Total elapsed time: 01h 55m 03s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |160               |96                
conv_1_kernel     |3                 |3                 
conv_2_filter     |192               |96                
conv_2_kernel     |3                 |3                 
conv_3_filter     |144               |208               
conv_3_kernel     |3                 |3                 
dense1_units      |48                |96                
dense2_units      |192               |240               
learning_rate     |0.01              |0.001             

Epoch 1/3
898/898 [==============================] - 1553s 2s/step - loss: 2.7905 - accuracy: 0.2397 - val_loss: 1.8087 - val_accuracy: 0.2581
Epoch 2/3
898/898 [==============================] - 1552s 2s/step - loss: 1.8125 - accuracy: 0.2497 - val_loss: 1.80